In [4]:
import pandas as pd

def provide_account_activity(csv_path: str, account_id: str = None):
    # Read CSV
    df = pd.read_csv(csv_path)
    return df[df['account_id'] == account_id] if account_id else None


if __name__ == "__main__":
    # Example usage
    csv_file_path = "large_call_center_memos.csv"
    print(provide_account_activity(csv_file_path,account_id='e3dce91d-ea4f-4d97-9514-bdeb43dacd04'))

     Memo ID                            account_id        Date Shortcode  \
115      115  e3dce91d-ea4f-4d97-9514-bdeb43dacd04  2025-03-08  NPS ACCT   
147      147  e3dce91d-ea4f-4d97-9514-bdeb43dacd04  2024-10-21      ACCT   
251      251  e3dce91d-ea4f-4d97-9514-bdeb43dacd04  2025-01-23   PIN RST   
273      273  e3dce91d-ea4f-4d97-9514-bdeb43dacd04  2025-02-13   ECS RVS   
425      425  e3dce91d-ea4f-4d97-9514-bdeb43dacd04  2024-12-14   ECS BLK   
446      446  e3dce91d-ea4f-4d97-9514-bdeb43dacd04  2025-03-20      DISP   
521      521  e3dce91d-ea4f-4d97-9514-bdeb43dacd04  2024-12-21   ECS BLK   
543      543  e3dce91d-ea4f-4d97-9514-bdeb43dacd04  2025-01-19   PIN RST   
557      557  e3dce91d-ea4f-4d97-9514-bdeb43dacd04  2025-04-16   PND DOC   
591      591  e3dce91d-ea4f-4d97-9514-bdeb43dacd04  2025-01-02   PIN RST   
633      633  e3dce91d-ea4f-4d97-9514-bdeb43dacd04  2024-11-01   ECS BLK   
675      675  e3dce91d-ea4f-4d97-9514-bdeb43dacd04  2024-11-27   FEE WVR   
731      731

In [5]:
pip install google-cloud-aiplatform

INFO: pip is looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 61.6 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


In [8]:
import vertexai


PROJECT_ID = "ai-experimentation-428115"  # Replace with your actual project ID
REGION = "us-central1"  # Choose your desired region

vertexai.init(project=PROJECT_ID, location=REGION)

from vertexai.language_models import TextEmbeddingModel, TextEmbeddingInput

def embed_text(csv_path="banking_call_center_memos.csv",
               model_name="text-embedding-005",
               task_type=None,
               title=None,
               output_dimensionality=None,
               auto_truncate=True):
    """Generates text embeddings for a list of texts using a specified model."""
    df = pd.read_csv(csv_path)
    model = TextEmbeddingModel.from_pretrained(model_name)
    inputs = [
        TextEmbeddingInput(
            text=row.get('Shortcode', '')+row.get('FullForm', '')+row.get('Description', ''),
            task_type=task_type,
            title=title
        ) for _, row in df.iterrows()
    ]
    kwargs = {}
    if output_dimensionality is not None:
        kwargs["output_dimensionality"] = output_dimensionality
    if auto_truncate is not True:
        kwargs["auto_truncate"] = auto_truncate

    embeddings = model.get_embeddings(inputs, **kwargs)
    df["embeddings"] = [embedding.values for embedding in embeddings]
    return df

embed_text(
    csv_path="banking_call_center_shortcodes.csv",
    model_name="text-embedding-005",
    task_type=None,
    title=None,
    output_dimensionality=None,
    auto_truncate=True
)

GoogleAuthError: 
Unable to authenticate your request.
Depending on your runtime environment, you can complete authentication by:
- if in local JupyterLab instance: `!gcloud auth login` 
- if in Colab:
    -`from google.colab import auth`
    -`auth.authenticate_user()`
- if in service account or other: please follow guidance in https://cloud.google.com/docs/authentication

In [11]:
!gcloud auth application-default login

/bin/bash: gcloud: command not found


In [2]:
from vertexai import rag
import vertexai, pandas as pd

PROJECT_ID = "ai-experimentation-428115"  # Replace with your actual project ID
REGION = "us-central1"  # Choose your desired regio
vertexai.init(project=PROJECT_ID, location=REGION)

# ── 2·1  Create the corpus (Vector DB is provisioned for you)
embed_cfg = rag.RagEmbeddingModelConfig(
    vertex_prediction_endpoint=rag.VertexPredictionEndpoint(
        publisher_model="publishers/google/models/text-embedding-005"   # Gemini‑era embedder
    )
)
corpus = rag.create_corpus(
    display_name="memo_glossary_corpus",
    backend_config=rag.RagVectorDbConfig(rag_embedding_model_config=embed_cfg),
)
# ── 2·2  Import the glossary file
rag.import_files(
    corpus.name,
    paths=["gs://banking_shortcodes/banking_call_center_shortcodes.csv"],
    transformation_config=rag.TransformationConfig(       # Optional chunking
        chunking_config=rag.ChunkingConfig(chunk_size=256, chunk_overlap=64)
    ),
)


ModuleNotFoundError: No module named 'vertexai'

In [25]:
import re
import pandas as pd

csv_file_path = "/Users/kishorekota/git/agentic_app/llm_summary_with_finetuning/large_call_center_memos.csv"
memo_df = pd.read_csv(csv_file_path)

memo_df = memo_df[memo_df['account_id'] == 'e3dce91d-ea4f-4d97-9514-bdeb43dacd04']

memo_df.sort_values(by='Date', ascending=False, inplace=True)
memo_text = pd.concat([memo_df['Date'], memo_df['Memo']], axis=1).to_string(index=False)

CODES = r"\b[A-Z]{3,5}\b"   # tweak to your pattern

def lookup_definitions(memo_text:str, top_k:int=1):
    # Extract unique codes from the memo
    hits = set(re.findall(CODES, memo_text))

    print(hits)
    
    if not hits:
        return {}
    # Run a single RAG retrieval query per code
    defs = {}
    for code in hits:
        resp = rag.retrieval_query(
            rag_resources=[rag.RagResource(rag_corpus=corpus.name)],
            text=code,
            rag_retrieval_config=rag.RagRetrievalConfig(top_k=top_k)
        )
        print(" response ",resp)
        print(" response context ",resp.contexts)
        defs[code] = resp.contexts[0].text if resp.contexts else None  # Access the `text` attribute of the first context
        # for context in resp.contexts:
        #     print(context.text)
        #     defs[code] = context.text  # Access the `text` attribute of each context
        #     break  # Only take the first match
    return defs






In [ ]:
import pandas as pd
import re

def get_shortcode_description(memo_text:str):
    """
    Looks up the description of a given shortcode from the CSV file.

    Args:
        csv_path (str): Path to the CSV file containing shortcodes and descriptions.
        shortcode (str): The shortcode to look up.

    Returns:
        str: The description of the shortcode, or a message if not found.
    """
    CODES = r"\b[A-Z]{3,6}\b"   # tweak to your pattern
    hits = set(re.findall(CODES, memo_text))
    csv_file_path = "/Users/kishorekota/git/agentic_app/llm_summary_with_finetuning/banking_call_center_shortcodes.csv"
    # Load the CSV file into a DataFrame
    df = pd.read_csv(csv_file_path)
    
    defs = {}
    for code in hits:

        result = df[df['Shortcode'] == code] 
        print("--------")
        print(result)
        if result.empty:
            defs[code] = "No description found"
        else:
            # Assuming the first match is the one you want
            # You can modify this logic if you need to handle multiple matches differently
            result = result.iloc[0]
            print("$$$$$",result)
            # Access the 'Description' column
            print("@@@",result['FullForm'])
            defs[code] = result['FullForm']
            print("%%%%", defs)

    return defs

defs = get_shortcode_description(memo_text)

#print(defs)


--------
Empty DataFrame
Columns: [Shortcode, FullForm, Description]
Index: []
--------
  Shortcode FullForm                                    Description
1      ACCT  Account  Refers to any bank account (savings/checking)
$$$$$ Shortcode                                               ACCT
FullForm                                             Account
Description    Refers to any bank account (savings/checking)
Name: 1, dtype: object
@@@ Account
%%%% {'TRF': 'No description found', 'ACCT': 'Account'}
--------
Empty DataFrame
Columns: [Shortcode, FullForm, Description]
Index: []
--------
Empty DataFrame
Columns: [Shortcode, FullForm, Description]
Index: []
--------
Empty DataFrame
Columns: [Shortcode, FullForm, Description]
Index: []
--------
Empty DataFrame
Columns: [Shortcode, FullForm, Description]
Index: []
--------
Empty DataFrame
Columns: [Shortcode, FullForm, Description]
Index: []
--------
Empty DataFrame
Columns: [Shortcode, FullForm, Description]
Index: []
--------
Empty DataFra

In [31]:
from vertexai.generative_models import GenerativeModel, Tool

# Build a retrieval *tool* so Gemini can pull extra facts on its own
retrieval_tool = Tool.from_retrieval(
    retrieval=rag.Retrieval(
        source=rag.VertexRagStore(
            rag_resources=[rag.RagResource(rag_corpus=corpus.name)],
            rag_retrieval_config=rag.RagRetrievalConfig(top_k=3)
        )
    )
)
summarizer = GenerativeModel(
    model_name="gemini-2.0-flash-001",
    tools=[retrieval_tool]
)

def summarize_call(memo_text:str):
    defs = get_shortcode_description(memo_text)
    print(" defs ",defs)
    glossary_block = "\n".join(f"{k} = {v}" for k,v in defs.items())
    prompt = f"""
### Glossary
{glossary_block}

### Memo
{memo_text}

### Task
Write a plain‑English call summary for the account rep. 
Use the glossary meanings, avoid jargon, and keep it under 120 words.
"""
    print(prompt)
    return summarizer.generate_content(prompt).text

print(memo_text)
summarize_call(memo_text)

      Date                                                                                              Memo
2025-04-16  2025-04-16 | PND DOC | Rep: REP143 | Note: Collection notice acknowledged; partial payment made.
2025-03-24                                                                2025-03-24 | LMT INC | Rep: REP228
2025-03-20                        2025-03-20 | DISP | Rep: REP619 | Note: Fraud alert placed on the account.
2025-03-09                                                                2025-03-09 | FVR CHK | Rep: REP755
2025-03-08                                                               2025-03-08 | NPS ACCT | Rep: REP522
2025-02-16                                                                2025-02-16 | BAL TRF | Rep: REP420
2025-02-14                                                                2025-02-14 | BAL TRF | Rep: REP350
2025-02-13                                                                2025-02-13 | ECS RVS | Rep: REP443
2025-01-23         

IndexError: single positional indexer is out-of-bounds